In [1]:
import requests
import pandas as pd
import chdb


### Lab 2.1

In [2]:
# Lab 2.1
file_path="https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/223/282.json"
response=requests.get(file_path)
df_matches=pd.json_normalize(response.json(), sep="_")
print(df_matches.iloc[0])

match_id                                                                    3943077
match_date                                                               2024-07-15
kick_off                                                               04:15:00.000
home_score                                                                        1
away_score                                                                        0
match_status                                                              available
match_status_360                                                        unscheduled
last_updated                                             2024-07-15T15:50:08.671355
last_updated_360                                                               None
match_week                                                                        6
competition_competition_id                                                      223
competition_country_name                                              South 

In [3]:
query="""
DESCRIBE Python(df_matches)
SETTINGS describe_compact_output=1
"""
query=chdb.query(query, "DataFrame")
print(query)


                              name    type
0                         match_id   Int64
1                       match_date  String
2                         kick_off  String
3                       home_score   Int64
4                       away_score   Int64
5                     match_status  String
6                 match_status_360  String
7                     last_updated  String
8                 last_updated_360  String
9                       match_week   Int64
10      competition_competition_id   Int64
11        competition_country_name  String
12    competition_competition_name  String
13                season_season_id   Int64
14              season_season_name  String
15          home_team_home_team_id   Int64
16        home_team_home_team_name  String
17      home_team_home_team_gender  String
18       home_team_home_team_group  String
19            home_team_country_id   Int64
20          home_team_country_name  String
21              home_team_managers  String
22         

#### lab 2.1 debrief - Argentina as home team

In [4]:

query="""
    SELECT home_team_home_team_name as team, count() as count
    FROM Python(df_matches)
    GROUP BY home_team_home_team_name
    ORDER BY count DESC
"""
query=chdb.query(query, "DataFrame")
print(query)

             team  count
0       Argentina      5
1         Uruguay      4
2        Colombia      3
3          Brazil      2
4         Ecuador      2
5            Peru      2
6          Canada      2
7          Mexico      2
8       Venezuela      2
9   United States      2
10       Paraguay      1
11         Panama      1
12          Chile      1
13        Jamaica      1
14        Bolivia      1
15     Costa Rica      1


#### lab 2.1 debrief - Argentina as away team

In [5]:

query="""
    SELECT away_team_away_team_name as team, count() as count
    FROM Python(df_matches)
    GROUP BY away_team_away_team_name
    ORDER BY count DESC
"""
query=chdb.query(query, "DataFrame")
print(query)

             team  count
0          Canada      4
1          Panama      3
2        Colombia      3
3        Paraguay      2
4         Uruguay      2
5           Chile      2
6          Brazil      2
7         Jamaica      2
8         Ecuador      2
9         Bolivia      2
10      Venezuela      2
11     Costa Rica      2
12           Peru      1
13         Mexico      1
14      Argentina      1
15  United States      1


In [6]:
# at a glance of the data
df_matches[['home_team_home_team_name', 'away_team_away_team_name']]

,home_team_home_team_name,away_team_away_team_name
0,Argentina,Colombia
1,Canada,Uruguay
2,Uruguay,Colombia
3,Argentina,Canada
4,Colombia,Panama
5,Uruguay,Brazil
6,Argentina,Peru
7,Uruguay,Panama
8,Venezuela,Canada
9,Argentina,Ecuador


In [7]:
query = chdb.query("""
SELECT team_name, COUNT(*) AS total_matches
FROM (
    SELECT home_team_home_team_name AS team_name
    FROM Python(df_matches)

    UNION ALL

    SELECT away_team_away_team_name AS team_name
    FROM Python(df_matches)
) AS all_teams
GROUP BY team_name
ORDER BY total_matches DESC;
""", "DataFrame")

print(query)

        team_name  total_matches
0       Argentina             10
1         Uruguay              8
2        Colombia              6
3          Brazil              4
4         Ecuador              4
5            Peru              4
6          Canada              4
7          Mexico              4
8       Venezuela              4
9   United States              4
10       Paraguay              2
11         Panama              2
12          Chile              2
13        Jamaica              2
14        Bolivia              2
15     Costa Rica              2


### Lab 2.3

In [8]:
file_path_amz="s3://datasets-documentation/amazon_reviews/*2014*.parquet"

In [9]:
query=f"""
SELECT _file, _path
FROM s3('{file_path_amz}', One)
SETTINGS output_format_pretty_row_numbers=0
"""
chdb.query(query, "PrettyCompact")

┌─_file──────────────────────────────┬─_path────────────────────────────────────────────────────────────────────┐
│ amazon_reviews_2014.snappy.parquet │ datasets-documentation/amazon_reviews/amazon_reviews_2014.snappy.parquet │
└────────────────────────────────────┴──────────────────────────────────────────────────────────────────────────┘

In [10]:
query=f"""
DESCRIBE s3('{file_path_amz}')
SETTINGS describe_compact_output=1
"""
chdb.query(query, "PrettyCompact")

    ┌─name──────────────┬─type─────────────┐
 1. │ review_date       │ Nullable(UInt16) │
 2. │ marketplace       │ Nullable(String) │
 3. │ customer_id       │ Nullable(UInt64) │
 4. │ review_id         │ Nullable(String) │
 5. │ product_id        │ Nullable(String) │
 6. │ product_parent    │ Nullable(UInt64) │
 7. │ product_title     │ Nullable(String) │
 8. │ product_category  │ Nullable(String) │
 9. │ star_rating       │ Nullable(UInt8)  │
10. │ helpful_votes     │ Nullable(UInt32) │
11. │ total_votes       │ Nullable(UInt32) │
12. │ vine              │ Nullable(Bool)   │
13. │ verified_purchase │ Nullable(Bool)   │
14. │ review_headline   │ Nullable(String) │
15. │ review_body       │ Nullable(String) │
    └───────────────────┴──────────────────┘

In [11]:
query=f"""
SELECT product_title, sum(verified_purchase) as total_valid_purchase
FROM s3('{file_path_amz}')
WHERE verified_purchase=TRUE
GROUP BY product_title
ORDER BY total_valid_purchase DESC
LIMIT 10
"""
chdb.query(query, "PrettyCompact")

    ┌─product_title───────────────────────────────────────────────┬─total_valid_purchase─┐
 1. │ Candy Crush Saga                                            │                25078 │
 2. │ The Fault in Our Stars                                      │                22139 │
 3. │ The Secret Society® - Hidden Mystery                        │                21216 │
 4. │ Flappy Wings (not Flappy Bird)                              │                19879 │
 5. │ Google Chromecast HDMI Streaming Media Player               │                17030 │
 6. │ Kindle Fire HDX 7", HDX Display (Previous Generation - 3rd) │                14558 │
 7. │ The Goldfinch: A Novel (Pulitzer Prize for Fiction)         │                14020 │
 8. │ Amazon.com eGift Cards                                      │                14017 │
 9. │ Gone Girl: A Novel                                          │                13961 │
10. │ Escape The Titanic                                          │                12702 │

### Lab 2.5 Remote

In [12]:

query2 = """
SELECT
    toStartOfDay(date-1)::Date32 AS day,
    sumIf(count, project = 'openai') AS openai_downloads,
    sumIf(count, project = 'scikit-learn') AS sklearn_downloads,
    round(
        sumIf(count, project = 'openai') / nullIf(sumIf(count, project = 'scikit-learn'), 0),
        4
    ) AS openai_to_sklearn_ratio
FROM remoteSecure(
    'sql-clickhouse.clickhouse.com',
    'pypi.pypi_downloads_per_day',
    'play'
)
GROUP BY day
ORDER BY day DESC
LIMIT 1
"""

df = chdb.query(query2, "DataFrame")
df

,day,openai_downloads,sklearn_downloads,openai_to_sklearn_ratio
0,2025-11-14,3251693,4031215,0.8066


### Lab 2.6 Streaming queries to speed up large size

In [13]:

data = {
    'trip_id': [1199999922, 1199999929],
    'pickup_datetime': ['2015-07-07 20:38:03', '2015-07-07 20:45:08'],
    'dropoff_datetime': ['	2015-07-07 20:50:13', '2015-07-07 21:06:52'],
    'pickup_longitude': [-73.97434997558594, -73.96578216552734],
    'pickup_latitude': [40.7622184753418, 40.75436019897461],
    'dropoff_longitude': [-73.98577117919922, -73.85907745361328],
    'dropoff_latitude': [40.73539352416992, 40.728702545166016],
    'passenger_count': [1, 1],
    'trip_distance': [2.29, 6.8],
    'fare_amount': [10.5,22.5],
    'extra': [0.5, 0.5],
    'tip_amount': [1.2, 0],
    'tolls_amount': [0, 0],
    'total_amount': [13, 23.8],
    'payment_type': ['CSH', 'CRE'],
    'pickup_ntaname': ['Midtown-Midtown South', 'Turtle Bay-East Midtown'],
    'dropoff_ntaname': ['Gramercy', 'Rego Park']
}

partial_taxi_df = pd.DataFrame(data)


In [14]:
    
parquet_path = "./taxi_trips.parquet"
s3_path = "s3://learn-clickhouse/nyc-taxi/trips0.tsv.gz"

taxi_query = """
WITH 
    s3_data AS (
        SELECT * FROM s3('{s3_path}')
    ),

    df_data AS (
        SELECT * FROM Python(partial_taxi_df)

    )

SELECT * 
FROM s3_data
FULL JOIN df_data ON df_data.trip_id = COALESCE(s3_data.trip_id)
""".format(s3_path=s3_path)


taxi_df = chdb.query(taxi_query, output_format="DataFrame")
taxi_df

,trip_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,trip_distance,fare_amount,...,df_data.passenger_count,df_data.trip_distance,df_data.fare_amount,df_data.extra,df_data.tip_amount,df_data.tolls_amount,df_data.total_amount,df_data.payment_type,df_data.pickup_ntaname,df_data.dropoff_ntaname
0,1.200000e+09,1.436313e+09,1.436314e+09,-73.959549,40.759048,-73.992340,40.749142,2.0,2.59,14.5,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
1,1.200000e+09,1.436315e+09,1.436316e+09,-73.781944,40.644711,-73.784576,40.666264,1.0,2.4,9,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
2,1.200000e+09,1.436319e+09,1.436320e+09,-74.008209,40.721554,-73.967491,40.770313,1.0,5.13,20,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
3,1.200000e+09,1.436324e+09,1.436325e+09,-74.000526,40.734367,-73.992073,40.749302,1.0,1.2,7,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
4,1.200000e+09,1.436328e+09,1.436329e+09,-73.988579,40.718754,-73.981995,40.743259,5.0,2.17,8.5,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000657,1.220000e+09,1.441625e+09,1.441625e+09,-73.984200,40.725109,-73.981590,40.732800,2.0,0.66,4.5,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
1000658,1.220000e+09,1.441629e+09,1.441630e+09,-73.984978,40.761131,-74.009666,40.702011,1.0,6.9,21.5,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
1000659,1.220000e+09,1.441631e+09,1.441634e+09,-73.792778,40.661274,-73.985840,40.760002,1.0,16.4,52.5,...,0,0.00,0.0,0.0,0.0,0,0.0,,,
1000660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,1,2.29,10.5,0.5,1.2,0,13.0,CSH,Midtown-Midtown South,Gramercy


In [15]:
from chdb import session
import time
  
start_time = time.time()
sess = session.Session()
stream_result = sess.send_query("SELECT * FROM Python(taxi_df)", "CSV")
stream_result.close()
end_time = time.time()
print(f"Streaming query execution time: {end_time - start_time:.4f} seconds")

start_time = time.time()
stream_result = chdb.query("SELECT * FROM Python(taxi_df)", "CSV")
end_time = time.time()
print(f"Regular query execution time: {end_time - start_time:.4f} seconds")

Streaming query execution time: 0.0668 seconds
Regular query execution time: 0.2444 seconds
